# Temporal Difference (TD) Learning 

TD learning was a breackthroug in solving the problem of reward prediction. It simplifies the complex reasoning about the future.
* TD combines the current reward and its prediction of the reward at the next moment in time.
* At the next moment, the new prediction is compared against what it is expected to be.
* The algorithm computes the differences to tune the old prediction with the new prediction
Minimizing those differens along time, that is, matching predictions to reality, the whole prediction mechanism continuously improves.

At the same time, late 80s and early 90s, neuroscience was trying to understand the behaviour of dopamine neurons. Dopamine neurones are clustered in the midbrain, but from there transmit to other areas distributing relevant messages. It was clear for these neurons that:
* Firing had some relationship to reward
* Responses depended also on the sensory input
* Responses changed as the animals got experience in a given task

Some researchers were in the both sides of Neuroscience and AI and noticed that:
* Responses in some dopamine neurons represented reward prediction errors
* The brain uses a TD learning algorithm

Reward prediction error theory of dopamine has become one of the most successful quantitative theories in neuroscience

https://deepmind.com/blog/article/Dopamine-and-temporal-difference-learning-A-fruitful-relationship-between-neuroscience-and-AI

## Recap 

![agent_environment_interaction_in_mdp](assets/agent_environment_interaction_in_mdp.png)
*The agent environment interaction in a MDP (http://incompleteideas.net/book/RLbook2020.pdf#page=70)*

**Run** (Episodic or Continuous)
$$S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2, \ldots$$

**Return** (Discounted sum of future rewards)
$$G_t := \sum_{k=0}^\infty \gamma^k R_{t+k+1}$$

**Policy**: Action to take at each state

**Value function** (state or state-action): estimates the expected future return for each state or state-action pair for a given policy
$$v_\pi(s) := E_\pi\left [G_t | S_t = s\right]$$

**Bellman equations**
$$v_\pi(s) = \sum_a \pi(a|s) \sum_{s'}\sum_r p(s',r|s,a)\left( r + \gamma v_\pi(s') \right) $$
$$q_\pi(s, a) = \sum_{s'} \sum_r p(s',r|s,a)\left( r + \gamma \sum_{a'} \pi(a'|s') q_\pi(s', a')\right) $$

**Bellman optimality equations**
$$v_*(s) = \max_a \sum_{s', r} p(s',r|s,a)\left( r + \gamma v_*(s') \right) $$
$$q_*(s, a) = \sum_{s', r} p(s',r|s,a)\left( r + \gamma \max_{a'} q_*(s', a')\right) $$

**Policy iteration**: Recursive policy evaluation and policy improvement.
$$\pi_0 \rightarrow v_{\pi_0} \rightarrow \pi_1 \rightarrow v_{\pi_1} \rightarrow \ldots \rightarrow \pi_* \rightarrow v_* $$

**Value iteration**: One update operation instead of separate policy evaluation and policy improvement
$$v_{k+1}(s) := \max_a \sum_{s', r} p(s',r|s,a)\left( r + \gamma v_k(s') \right) $$

**Generalized policy iteration (GPI)**: general idea of interacting between policy evaluation and policy improvement independent of "how much" of each one is done

#### Monte Carlo methods

* Sample-based method
* Valid for unavailable or complex environments
* Estimate by averaging over multiple returns
$$ V(S_t) \leftarrow V(S_t) + \alpha (G_t - V(S_t))$$
* Updates are done after the end of an episode, so it is used on episodic tasks

## Import libraries 

In [ ]:
from abc import ABCMeta, abstractmethod

import gym
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as py
from scipy import stats


py.init_notebook_mode(connected=True)  # Comment for Google Colab

## TD Prediction

In the prediction problem, the goal is to estimate

$$v_\pi(s) := E[G_t | S_t = s], $$

that is the expected return starting from a given state.

The following are the sample based approximations Monte Carlo and Temporal Difference:

$$\begin{align}
V(S_t) \leftarrow{ }& V(S_t) + \alpha (G_t - V(S_t)) \quad \text{(Monte Carlo)} \\
V(S_t) \leftarrow{ }& V(S_t) + \alpha (R_{t+1} + \gamma V(S_{t+1}) - V(S_t)) \quad \text{(Temporal Difference)}
\end{align}$$

Thus, for TD, an approximation is done on

$$ G_t \approx R_{t+1} + \gamma V(S_{t+1}) $$

On TD algorithms, to estimate the value $V(S_t)$ of a state, the estimate of the next state $V(S_{t+1})$ is used. The update rule above is called *TD(0)* or *one-step TD* as it is updated looking only at the following state's estimation.

![tabular_td0_for_estimating_v](assets/tabular_td0_for_estimating_v.png)

In [ ]:
def tabular_td_episode(v, env, pi, alpha, gamma):
    s = env.reset()
    done = False
    while not done:
        a = pi.get_action(s)
        s_prime, r, done, _ = env.step(a)
        v[s] += alpha * (r + gamma * v[s_prime] - v[s])
        s = s_prime
    v[s] = 0

Let's apply TD(0) to predict the values of the states on the FrozenLake environment of size 8x8. The aim of the environment is to move over a grid world until goal is reached (win) or falled into a hole (lose). Moreover, as the world is frozen, trying to go to a direction does not imply it is fulfilled.

In [ ]:
env = gym.make('FrozenLake8x8-v0')
env.reset()
env.render()

Actions are 0:Left, 1:Down, 2:Right, 3:Up. Playing for some steps, can be checked the environment and its randomness.

In [ ]:
print(env.step(2))
env.render()

To predict values of states for a given policy, it is required a policy to evaluate. In this case, the selected policy that either goes to South or East, both with equal probability 0.5. Other actions are not taken, i.e., have probability 0.

In [ ]:
class SouthEastPolicy:
    def __init__(self):
        pass
    
    def get_action(self, s):
        return np.random.choice([1, 2])

Let us run the previously defined TD(0) algorithm on the Frozen Lake environment to estimate the values of states given the policy that randomly chooses Down or Right.

In [ ]:
def plot_frozen_lake_state_values(seed=2):
    np.random.seed(seed)
    n_episodes = 10000
    env = gym.make("FrozenLake8x8-v0")
    env.reset()
    env.render()
    v = np.zeros(8 * 8)
    pi = SouthEastPolicy()
    for _ in range(n_episodes):
        tabular_td_episode(v, env, pi, alpha=0.3, gamma=1)
    fig = go.Figure(data=go.Heatmap(
                    z=v.reshape(8, 8)))
    fig.update_layout(
        yaxis_autorange="reversed",
        title="State values on 8x8 Frozen Lake for Down-Right random policy")
    fig.show()

In [ ]:
plot_frozen_lake_state_values()

Observe how states that are down and left have the lowest values. Remember that this values are for the fixed policy that always chooses either Down or Right. That area is full of Holes, which means the probability of reaching the Goal from those areas is not probable. On the other hand, states that are on the right have the highest values, as the policy goes to the Goal from there.

## Advantages of TD  Prediction Methods

TD methods *bootstrap*, that is, they update estimates based on other estimates.
* TD methods do not require a model of the environment as in Dynamic Programming.
* TD methods are on-line and incremental, unlike Monte Carlo. Waiting until the end of a possibly long episode to update the values be critical. Moreover, continuous tasks that are not naturally splited in episodes.
* TD(0) converges to $v_\pi$ on the mean for a constant step-size if it is sufficiently small, and with probability 1 if the step-size parameter decreases according to the usual stochastic approximation conditions $\sum_{n=1}^\infty \alpha_n(a) = \infty$ and $\sum_{n=1}^\infty \alpha_n^2(a) < \infty$ (f.e. $\alpha_n(a)=1/n$).
* In practice, the convergence is usually faster than Monte Carlo methods.

### Random Walk Experiment

In [ ]:
class RandomWalk(gym.Env):
    def __init__(self, states_to_terminal=3):
        super(RandomWalk, self).__init__()
        self.nS = 2 * states_to_terminal + 1
        self.initial_s = states_to_terminal
        self.s = None

    def reset(self):
        self.s = self.initial_s
        return self.s
        
    def step(self, a=None):
        if self.s in [0, self.nS - 1]:
            return self.s, 0, True, {}
        self.s += np.random.choice([-1, 1], p=[0.5, 0.5])
        if self.s == 0:
            reward, done = 0, True
        elif self.s == self.nS - 1:
            reward, done = 1, True
        else:
            reward, done = 0, False
        return self.s, reward, done, {}
    
    def render(self):
        state = np.full(self.nS, 'O')
        state[self.s] = 'X'
        print(state)

In [ ]:
env = RandomWalk()
env.reset()
done = False
env.render()
while done is False:
    _, r, done, _ = env.step()
    print(f'Reward: {r}')
    env.render()

To compare the performance of TD(0) against Monte Carlo, let's define a function for MC update.

In [ ]:
def mc_rw_episode(v, env, pi, alpha):
    s = env.reset()
    done = False
    s_hist = [s]
    while not done:
        a = pi.get_action(s)
        s, r, done, _ = env.step(a)
        s_hist.append(s)
    v[s] = 0
    for s in s_hist[:-1]:
        v[s] += alpha * (r - v[s])

Then run a experiment on Random Walk environment to:
* See the evolution of state values with TD(0)
* Compare TD(0) and MC for different $\alpha$ values

In [ ]:
def plot_random_walk_td_vs_mc(seed=2):
    np.random.seed(seed=seed)
    n_episodes = 200
    env = RandomWalk()
    v_true = [0, 1/6, 2/6, 3/6, 4/6, 5/6, 0]
    v = [0, .5, .5, .5, .5, .5, 0]
    pi = SouthEastPolicy()  # Policy has no effect
    v_record = np.zeros((n_episodes + 1, 7))
    v_record[0, :] = v
    for i in range(1, n_episodes + 1):
        tabular_td_episode(v, env, pi, alpha=0.1, gamma=1)
        v_record[i, :] = v

    fig = go.Figure()
    fig.add_trace(go.Scatter(
            y=v_true[1:6],
            x=np.arange(1, 6),
            mode='lines',
            name=f'True'))
    episodes = [0, 1, 10, 100]
    for ep in episodes:
        fig.add_trace(go.Scatter(
            y=v_record[ep, 1:6],
            x=np.arange(1, 6),
            mode='lines',
            name=f'{ep}'))
    fig.update_layout(
        title='Estimated values after a number of episodes on a single run of TD(0)',
        yaxis_title='Estimated value',
        xaxis_title='State',
        yaxis_range=None,
        xaxis_tickmode='linear')
    fig.show()
    
    fig_rmse = go.Figure()
    n_runs = 100
    n_episodes = 100
    v_true = [0, 1/6, 2/6, 3/6, 4/6, 5/6, 0]
    # TD runs
    for alpha in [0.05, 0.1, 0.15]:
        v_record = np.zeros((n_runs, n_episodes + 1, 7))
        for run in range(n_runs):
            v = [0, .5, .5, .5, .5, .5, 0]
            pi = SouthEastPolicy()  # Policy has no effect
            v_record[run, 0, :] = v
            for ep in range(1, n_episodes + 1):
                tabular_td_episode(v, env, pi, alpha=alpha, gamma=1)
                v_record[run, ep, :] = v
        rmse = np.sqrt(np.sum(np.power(v_record - v_true, 2), axis=2) / 5)
        rmse = np.mean(rmse, axis=0)
        fig_rmse.add_trace(go.Scatter(
            y=rmse,
            x=np.arange(n_episodes + 1),
            mode='lines',
            name=f'TD: {alpha}'))
    # MC runs
    for alpha in [0.01, 0.02, 0.03, 0.04]:
        v_record = np.zeros((n_runs, n_episodes + 1, 7))
        for run in range(n_runs):
            v = [0, .5, .5, .5, .5, .5, 0]
            pi = SouthEastPolicy()  # Policy has no effect
            v_record[run, 0, :] = v
            for ep in range(1, n_episodes + 1):
                mc_rw_episode(v, env, pi, alpha=alpha)
                v_record[run, ep, :] = v
        rmse = np.sqrt(np.sum(np.power(v_record - v_true, 2), axis=2) / 5)
        rmse = np.mean(rmse, axis=0)
        fig_rmse.add_trace(go.Scatter(
            y=rmse,
            x=np.arange(n_episodes + 1),
            mode='lines',
            line = dict(dash='dash'),
            name=f'MC: {alpha}'))
    fig_rmse.update_layout(
        title='RMSE of state values averaged over states over multiple runs',
        xaxis_title='Episode',
        yaxis_title='RMSE of state values')
    fig_rmse.show()
                
plot_random_walk_td_vs_mc(seed=6)

Results of TD(0) were better for any choice of the parameter $\alpha$ against any of the MC methods evaluated.

##  Optimality of TD(0)

TODO
ADD BATCH UPDATING INFO AND CODE EXAMPLE

Under batch updating:
* TD(0) converges deterministically to a single answer no matter the choice of a sufficiently small $\alpha$.
* Constant $\alpha$ MC also fulfills the condition, with another answer.

## Generalized Policy Iteration (GPI) 

Loop:
  1. Policy Evaluation ($\pi \rightarrow V$)
  2. Policy Improvement ($V \rightarrow \pi$)

Previously in MC:
* At the end of each episode PE and then PI
* PI without a full PE
* Evaluate + Improve each episode

Now, improve policy after just one Policy Evaluation step, with TD

Instead of looking to transitions between states $V(s)$, consider learning transitions between state-action to state-action $Q(s,a)$. This is SARSA.

## SARSA, on-policy TD control

![state_action_reward_run](assets/state_action_reward_run.png)
*http://incompleteideas.net/book/RLbook2020.pdf#page=153*

$$ Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha ( R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)) $$

Actions are taken according to the considered policy (based on Q-values).

Note the similarity to he TD update for the state values

$$ V(S_t) \leftarrow V(S_t) + \alpha \left( R_{t+1} + \gamma V(S_{t+1}) - V(S_t) \right) $$

This algorithm is just for policy evaluation with state-action values, but considering GPI it becomes a control algorithm.

In [ ]:
class SarsaAgent:
    def __init__(self, q=None, n_states=None, n_actions=None,
                 alpha=0.2, gamma=0.9, eps=0.1, seed=None):
        if seed is not None:
            np.random.seed(seed)
        if q is not None:
            self.q = np.array(q)
        elif n_states > 0 and n_actions > 0:
            self.q = np.zeros((n_states, n_actions))
        else:
            assert False
        self.gamma = gamma
        self.alpha = alpha
        self.eps = eps
        self.n_states = self.q.shape[0]
        self.n_actions = self.q.shape[1]
        
        self.last_s = None
        self.last_a = None
        self.n_steps = 0
        self.reward = 0
        
    def reset(self):
        self.q = np.zeros((self.n_states, self.n_actions))
    
    def init(self, s, a):
        self.last_s = s
        self.last_a = a
        self.n_steps = 0
        self.reward = 0

    def get_action(self, s, eps=None):
        eps = self.eps if eps is None else eps
        if np.random.rand() < (1 - eps):
            return np.random.choice(
                np.arange(self.n_actions)[self.q[s] == np.max(self.q[s])])
        else:
            return np.random.choice(np.arange(self.n_actions))
    
    def step(self, r, s):
        a = self.get_action(s)
        self.q[self.last_s, self.last_a] += self.alpha * (
            r + self.gamma * self.q[s, a] - self.q[self.last_s, self.last_a])
        self.last_s, self.last_a = s, a
        self.n_steps += 1
        self.reward += r
        return s, a

### SARSA Control in the Windy Grid World Environment

In [ ]:
LEFT = 2
DOWN = 1
RIGHT = 3
UP = 0

WORLD_HEIGHT = 7
WORLD_WIDTH = 10

WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]

START = [3, 0]
GOAL = [3, 7]


class WindyGridWorld(gym.Env):
    def __init__(self):
        super(WindyGridWorld, self).__init__()
        self.action_space = gym.spaces.Discrete(4)
        self.observation_space = gym.spaces.Discrete(27)
        self.world_height = WORLD_HEIGHT
        self.world_width = WORLD_WIDTH
        self.world_wind = WIND

        self.nA = 4
        self.nS = WORLD_HEIGHT * WORLD_WIDTH
        
        self.s = self._to_s(*START)

    def _to_s(self, row, col):
        return row * WORLD_WIDTH + col
    
    def _to_pos(self, s):
        return [s // WORLD_WIDTH, s % WORLD_WIDTH]
    
    def reset(self):
        self.s = self._to_s(*START)
        return self.s

    def step(self, a):
        i, j = self._to_pos(self.s)
        if a == UP:
            s_pos = [max(i - 1 - WIND[j], 0), j]
        elif a == DOWN:
            s_pos = [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), j]
        elif a == LEFT:
            s_pos = [max(i - WIND[j], 0), max(j - 1, 0)]
        elif a == RIGHT:
            s_pos = [max(i - WIND[j], 0), min(j + 1, WORLD_WIDTH - 1)]
        else:
            assert False
        self.s = self._to_s(*s_pos)
        reward = -1 if s_pos != GOAL else 0
        done = False if s_pos != GOAL else True
        return self.s, reward, done, {}
    
    def render(self):
        world = pd.DataFrame(np.full((WORLD_HEIGHT, WORLD_WIDTH), ' '))
        world.iloc[START[0], START[1]] = 'S'
        world.iloc[GOAL[0], GOAL[1]] = 'G'
        s_pos = self._to_pos(self.s)
        world.iloc[s_pos[0], s_pos[1]] = 'X'
        display(world)

In [ ]:
env = WindyGridWorld()
env.reset()
env.render()

In [ ]:
print(env.step(3))
env.render()

In [ ]:
def plot_windy_grid_world(seed=2):
    np.random.seed(2)
    env = WindyGridWorld()
    agent = SarsaAgent(q=np.zeros((env.nS, env.nA)), gamma=1, alpha=0.5)
    n_time_steps = 8001 
    time_steps = np.arange(n_time_steps)
    episodes = np.zeros(n_time_steps)
    time_step = 0
    while time_step < n_time_steps:
        s = env.reset()
        a = agent.get_action(s)
        agent.init(s, a)
        done = False
        while done is False:
            s, r, done, _ = env.step(a)
            _, a = agent.step(r, s)
        time_step += agent.n_steps
        episodes[time_step:n_time_steps] += 1

    fig = go.Figure()
    fig.update_layout(
        xaxis_title='Time Step',
        yaxis_title='Episodes solved')
    fig.add_trace(go.Scatter(
        y=episodes,
        mode='lines',
        name=f'Episodes'))
    fig.show()
    
    ep = pd.DataFrame(np.full((env.world_height, env.world_width), ' '))
    ep = ep.append(pd.DataFrame([env.world_wind], index=['Wind']))
    done = False
    s = env.reset()
    a = agent.get_action(s)
    agent.init(s, a)
    played_steps = 0
    while done is False and played_steps < 500:
        a = np.argmax(agent.q[s])
        s_pos = env._to_pos(s)
        ep.iloc[s_pos[0], s_pos[1]] = a
        s, r, done, _ = env.step(a)
        played_steps += 1

    s_pos = env._to_pos(s)
    ep.iloc[s_pos[0], s_pos[1]] = 'G'
    display(ep)

In [ ]:
plot_windy_grid_world()

Note: Applying a Monte Carlo method here is not easy. It would be an update until the episode is done, and if trapped in a state it could not finish. With Temporal Difference, estimates are updated during the episode, so the situation is avoided.

## Q-learning: Off-policy TD Control 

Most recent applications of Reinforcement Learning are based on Q-learning algorithm, such as the sounded Atari Games play learning.

![Q-learning](assets/q_learning.png)
*http://incompleteideas.net/book/RLbook2020.pdf#page=153*

The clear difference between Sarsa and Q-learning is on the update rule, which is defined as

$$ Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left ( R_{t+1} + \gamma \max_a Q(S_{t+1}, a) - Q(S_t, A_t) \right) $$

### Sarsa and Q-learning relation to Bellman equations

**Sarsa**: $ Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha ( R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)) $

$$ q_\pi(s,a)=\sum_{s',r} p(s',r|s,a) \left(r + \gamma \sum_{a'} \pi(a'|s') q_\pi(s',a') \right) $$

**Q-learning**: $ Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left ( R_{t+1} + \gamma \max_a Q(S_{t+1}, a) - Q(S_t, A_t) \right) $

$$ q_*(s,a) = \sum_{s',r} p(s',r|s,a) \left( r + \gamma \max_{a'} q_\pi(s',a') \right)$$

So Sarsa is based on the Bellman Equations for state-action values, whereas Q-learning is based on the Bellman Optimality Equations for state-action values. Thus:
* Sarsa -> Policy Iteration
* Q-learning -> Value Iteration

In [ ]:
class QLearnAgent:
    def __init__(self, q=None, n_states=None, n_actions=None,
                 alpha=0.2, gamma=0.9, eps=0.1, seed=None):
        if seed is not None:
            np.random.seed(seed)
        if q is not None:
            self.q = np.array(q)
        elif n_states > 0 and n_actions > 0:
            self.q = np.zeros((n_states, n_actions))
        else:
            assert False
        self.gamma = gamma
        self.alpha = alpha
        self.eps = eps
        self.n_states = self.q.shape[0]
        self.n_actions = self.q.shape[1]
        
        self.last_s = None
        self.last_a = None
        self.n_steps = 0
        self.reward = 0
        
    def reset(self):
        self.q = np.zeros((self.n_states, self.n_actions))
    
    def init(self, s, a):
        self.last_s = s
        self.last_a = a
        self.n_steps = 0
        self.reward = 0

    def get_action(self, s, eps=None):
        eps = self.eps if eps is None else eps
        if np.random.rand() < (1 - eps):
            return np.random.choice(
                np.arange(self.n_actions)[self.q[s] == np.max(self.q[s])])
        else:
            return np.random.choice(np.arange(self.n_actions))
    
    def step(self, r, s):
        a = self.get_action(s)
        a_max = self.get_action(s, eps=0)
        self.q[self.last_s, self.last_a] += self.alpha * (
            r + self.gamma * self.q[s, a_max] - self.q[self.last_s, self.last_a])
        self.last_s, self.last_a = s, a
        self.n_steps += 1
        self.reward += r
        return s, a

### Compare Windy Grid World with Q-learning and Sarsa

In [ ]:
def plot_windy_grid_world_q_learning_vs_sarsa(alpha_sarsa, alpha_q_learn, n_time_steps=8000, seed=2):
    np.random.seed(seed)
    env = WindyGridWorld()
    gamma = 1
    eps = 0.1
    n_time_steps = n_time_steps + 1
    fig = go.Figure()
    for i, alg in enumerate(['Sarsa', 'Q-learning']):
        episodes = np.zeros(n_time_steps)
        time_steps = np.arange(n_time_steps)
        if alg == 'Sarsa':
            agent = SarsaAgent(q=np.zeros((env.nS, env.nA)), alpha=alpha_sarsa, gamma=gamma, eps=eps)
        elif alg == 'Q-learning':
            agent = QLearnAgent(q=np.zeros((env.nS, env.nA)), alpha=alpha_q_learn, gamma=gamma, eps=eps)
        time_step = 0
        while time_step < n_time_steps:
            s = env.reset()
            a = agent.get_action(s)
            agent.init(s, a)
            done = False
            while done is False:
                s, r, done, _ = env.step(a)
                _, a = agent.step(r, s)
            time_step += agent.n_steps
            episodes[time_step:n_time_steps] += 1  
        fig.add_trace(go.Scatter(
            y=episodes,
            mode='lines',
            name=f'{alg}'))
    fig.update_layout(
        xaxis_title='Time Step',
        yaxis_title='Episodes solved')
    fig.show()

In [ ]:
plot_windy_grid_world_q_learning_vs_sarsa(alpha_sarsa=0.5, alpha_q_learn=0.5, n_time_steps=8000)

At first, both algorithms improve similarly and solve episodes at the same rate. Then, Q-learning improves considerably faster than Sarsa for the remaining time steps and achieves a better policy.

In fact, the slope of the curves determines that Q-learning has achieved the optimal policy (if slope remains constant, the policy is not improving as it solves episodes at a constant number of steps). However, Sarsa has not achieved the optimal policy yet for 8000 time steps.

As $\alpha=0.5$ could be high for Sarsa, let us run another experiment with a lower step and for longer time steps and see if it works better.

In [ ]:
plot_windy_grid_world_q_learning_vs_sarsa(alpha_sarsa=0.1, alpha_q_learn=0.5, n_time_steps=100_000)

In this experiment, we can see that this time Sarsa improves slower with the lower step $\alpha$ but it reaches the optimal policy. Actually, both Sarsa and Q-learning lines are parallel, which means that the policy the algorithms have reached is the same.

### Q-learning in Cliff World Environment

In [ ]:
UP, DOWN, LEFT, RIGHT = 0, 1, 2, 3
HEIGHT, WIDTH = 4, 12

START = [3, 0]
GOAL = [3, 11]
CLIFF = [[3, i] for i in range(1, 11)]


class CliffWalking(gym.Env):
    def __init__(self):
        super(CliffWalking, self).__init__()
        self.action_space = gym.spaces.Discrete(4)
        self.observation_space = gym.spaces.Discrete(HEIGHT * WIDTH)
        self.height = HEIGHT
        self.width = WIDTH

        self.nA = 4
        self.nS = HEIGHT * WIDTH
        
        self.s = self._to_s(*START)

    def _to_s(self, row, col):
        return row * WIDTH + col
    
    def _to_pos(self, s):
        return [s // WIDTH, s % WIDTH]
    
    def reset(self):
        self.s = self._to_s(*START)
        return self.s

    def step(self, a):
        i, j = self._to_pos(self.s)
        if a == UP:
            s_pos = [max(i - 1, 0), j]
        elif a == DOWN:
            s_pos = [max(min(i + 1, HEIGHT - 1), 0), j]
        elif a == LEFT:
            s_pos = [max(i, 0), max(j - 1, 0)]
        elif a == RIGHT:
            s_pos = [max(i, 0), min(j + 1, WIDTH - 1)]
        else:
            assert False
        if s_pos in CLIFF:
            reward = -100
            s_pos = START
        else:
            reward = -1
        self.s = self._to_s(*s_pos)
        done = False if s_pos != GOAL else True
        return self.s, reward, done, {}
    
    def render(self):
        world = pd.DataFrame(np.full((HEIGHT, WIDTH), ' '))
        world.iloc[START[0], START[1]] = 'S'
        world.iloc[GOAL[0], GOAL[1]] = 'G'
        for c in CLIFF:
            world.iloc[c[0], c[1]] = 'C'
        s_pos = self._to_pos(self.s)
        world.iloc[s_pos[0], s_pos[1]] = 'X'
        display(world)

In [ ]:
env = CliffWalking()
env.reset()
env.render()

In [ ]:
print(env.step(0))
env.render()

In [ ]:
def plot_cliff_walking():
    n_runs = 100
    n_episodes = 501
    env = CliffWalking()
    gamma = 1
    alpha = 0.5
    episodes = np.arange(n_episodes)

    def run_episodes(alg):
        sum_rewards = np.zeros(n_episodes)
        if alg == 'Sarsa':
            agent = SarsaAgent(q=np.zeros((env.nS, env.nA)), gamma=gamma, alpha=alpha, eps=0.1)
        elif alg == 'Q-learning':
            agent = QLearnAgent(q=np.zeros((env.nS, env.nA)), gamma=gamma, alpha=alpha, eps=0.1)
        for ep in episodes:
            s = env.reset()
            a = agent.get_action(s)
            agent.init(s, a)
            done = False
            while done is False:
                s, r, done, _ = env.step(a)
                _, a = agent.step(r, s)
            sum_rewards[ep] = agent.reward
        return sum_rewards, agent

    sum_rewards_q_learn, agents_q_learn = zip(*[run_episodes('Q-learning') for _ in range(n_runs)])
    sum_rewards_q_learn = np.mean(sum_rewards_q_learn, axis=0)
    sum_rewards_sarsa, agents_sarsa = zip(*[run_episodes('Sarsa') for _ in range(n_runs)])
    sum_rewards_sarsa = np.mean(sum_rewards_sarsa, axis=0)

    fig = go.Figure()

    ep = pd.DataFrame(np.full((env.height, env.width), ''))
    ep.iloc[-1, 0] = 'S'
    ep.iloc[-1, 1:-1] = 'C'
    ep.iloc[-1, -1] = 'G'
    for i, alg in enumerate(['Sarsa', 'Q-learning']):
        fig.add_trace(go.Scatter(
            y=sum_rewards_sarsa if alg =='Sarsa' else sum_rewards_q_learn,
            x=episodes,
            mode='lines',
            name=f'{alg}'))
        symbol = 's' if alg == 'Sarsa' else 'q'
        s = env.reset()
        done = False
        ep.iloc[-1, 0] += symbol
        played_steps = 0
        while done is False and played_steps < 500:
            if alg == 'Sarsa':
                a = stats.mode(
                    [agent.get_action(s, eps=0) for agent in agents_sarsa])[0]
            else:
                a = stats.mode(
                    [agent.get_action(s, eps=0) for agent in agents_q_learn])[0]
            s, r, done, _ = env.step(a)
            s_pos = env._to_pos(s)
            ep.iloc[s_pos[0], s_pos[1]] += symbol 
            played_steps += 1

    fig.update_layout(
        xaxis_title='Episode',
        yaxis_title='Episode sum of rewards',
        yaxis_range=[-100, np.max([sum_rewards_q_learn, sum_rewards_sarsa])])
    fig.show()
    display(ep)

In [ ]:
plot_cliff_walking()

## On-policy and off-policy

* On-policy: Learning "on" the target policy. The policy is learnt while it is executed.
* Off-policy: Two policies are involved, one that is being learnt and a second one which generates the data.
    * Target policy: The policy wanted to learn about.
    * Behavior policy: The one that chooses the actions to take, and thus, that gathers the data.
    
Sarsa is on-policy whereas Q-learning is off-policy.

## Summary

* TD is a sample-based method, as it is MC
* TD bootstraps, that is, updates an estimate towards another estimate
* TD has advantages over DP and MC:
    * Usually converges faster than MC
    * A model is not required as in DP
    * TD is fully online and incremental

* TD control is based on Bellman equations
    * Sarsa uses Bellman equations
    * Q-learning uses Bellman optimality equations

* Sarsa is an on-policy algorithm
* Q-learning is an off-policy algorithm

* When measuring the performance on-line, Sarsa can do better because on-line methods take account of their own exploration methods.